In [1]:
import pandas as pd

In [2]:
df = pd.read_json("../data/인터뷰/ckmk_d_bm_f_n_169087.json")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, info to answer
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   version      3 non-null      int64 
 1   dataSet      3 non-null      object
 2   rawDataInfo  2 non-null      object
dtypes: int64(1), object(2)
memory usage: 96.0+ bytes


In [4]:
df

,version,dataSet,rawDataInfo
info,1,"{'date': '20230116', 'occupation': 'BM', 'chan...",NaN
question,1,{'raw': {'text': '지원자님께서 가지고 계신 나만의 스트레스 해소법이 ...,"{'fileFormat': 'wav', 'fileSize': 664398, 'dur..."
answer,1,{'raw': {'text': '저는 저 혼자 시간을 보내는 것을 우선적으로 하고요...,"{'fileFormat': 'wav', 'fileSize': 1900238, 'du..."


In [5]:
# read_json을 통해 json파일을 로드했는데, 데이터 형태가 데이터프레임에 최적화되어 있지 않다.
# -> 답변의 데이터, 답변의 요약 데이터를 추출 -> kobart 모델을 이용해서 요약 학습
# 데이터 추출을 위해 json 파일을 dict 형태로 변환
import json

In [6]:
# 파일을 로드해서 dict 형태로 변환
# 방법 1
with open("../data/인터뷰/ckmk_d_bm_f_n_169087.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

In [7]:
# 방법 2
f2 = open("../data/인터뷰/ckmk_d_bm_f_n_169087.json", 'r', encoding='utf-8')
data2 = json.load(f2)

In [8]:
type(data)

dict

In [9]:
from pprint import pprint

In [10]:
print(data)

{'version': '1.0', 'dataSet': {'info': {'date': '20230116', 'occupation': 'BM', 'channel': 'MOCK', 'place': 'ONLINE', 'gender': 'FEMALE', 'ageRange': '-34', 'experience': 'NEW'}, 'question': {'raw': {'text': '지원자님께서 가지고 계신 나만의 스트레스 해소법이 있으신지 있으시다고 한다면 왜 그러한 방법으로 스트레스를 해소하려고 하셨는지 이야기해 주실 수 있으실까요', 'wordCount': 19}, 'emotion': [], 'intent': []}, 'answer': {'raw': {'text': '저는 저 혼자 시간을 보내는 것을 우선적으로 하고요. 예를 들어서 혼자 산책을 한다든지 아니면 혼자 카페를 간다든지 아니면 혼자 코인 노래방을 간다든지 해서 스트레스를 해소하는 편입니다. 물론 남들과 만나면서 시간을 보내는 것도 굉장히 좋아하지만요. 아무래도 내 혼자만의 시간이 있어야 어느 정도 생각 정리도 되고 그러면서 내 안에 있던 응어리도 절로 풀러지는 그런 것을 경험하게 되었습니다. 그리고 코인 노래방이나 이런 곳에 가면은 아무래도 제가 노래 부르는 것도 좋아하고 그리고 노래를 부르면서 그런 소리가 웅웅거리고 또 소리를 지른다는 거에 대해서 해소감도 있는 것 같고요. 그래서 저는 일단 혼자만의 시간을 보내는데 뭐 혼자서 노래방을 가거나 아니면 혼자서 카페를 가거나 그런 식으로 스트레스를 해소한다 라고 말씀을 드릴 수 있을 것 같습니다.', 'wordCount': 107}, 'emotion': [], 'intent': [{'text': '', 'expression': '', 'category': 'attitude'}], 'summary': {'text': '저는 혼자 시간을 보내는 것을 우선적으로 하며, 혼자 산책을 하거나 혼자 카페를 간다든지 또는 혼자 노래방을 간다든지 해서 스트레스를 해소하는 

In [11]:
pprint(data)

{'dataSet': {'answer': {'emotion': [],
                        'intent': [{'category': 'attitude',
                                    'expression': '',
                                    'text': ''}],
                        'raw': {'text': '저는 저 혼자 시간을 보내는 것을 우선적으로 하고요. 예를 들어서 '
                                        '혼자 산책을 한다든지 아니면 혼자 카페를 간다든지 아니면 혼자 코인 '
                                        '노래방을 간다든지 해서 스트레스를 해소하는 편입니다. 물론 남들과 '
                                        '만나면서 시간을 보내는 것도 굉장히 좋아하지만요. 아무래도 내 '
                                        '혼자만의 시간이 있어야 어느 정도 생각 정리도 되고 그러면서 내 '
                                        '안에 있던 응어리도 절로 풀러지는 그런 것을 경험하게 되었습니다. '
                                        '그리고 코인 노래방이나 이런 곳에 가면은 아무래도 제가 노래 부르는 '
                                        '것도 좋아하고 그리고 노래를 부르면서 그런 소리가 웅웅거리고 또 '
                                        '소리를 지른다는 거에 대해서 해소감도 있는 것 같고요. 그래서 저는 '
                                        '일단 혼자만의 시간을 보내는데 뭐 혼자서 노래방을 가거나 아니면 '

---
---
## kobart 복습 
1. 인터뷰 폴더 안에 있는 json 파일들을 로드 
    - 답변에 대한 데이터들을 하나의 리스트로 생성 
    - 답변 요약에 대한 데이터들을 하나의 리스트로 생성 
2. 위에서 생성된 데이터들 중
    - train 데이터는 처음 100개의 데이터
    - validation 데이터는 마지막 10개의 데이터
3. train, test 데이터를 DatasetDict 형태로 변환 
4. tokenizer와 model을 로드하여 데이터를 학습하고 검증 
    - input의 최대 사이즈 : 512
    - output의 최대 사이즈 : 256
5. test 데이터를 생성 : 원본 리스트의 200번째 데이터를 이용하여 요약을 생성

In [12]:
# json 파일의 목록을 생성 
import os 
from glob import glob

In [13]:
file_path = "../data/인터뷰/"
file_list = os.listdir(file_path)

In [14]:
# 또는
file_list2 = glob("../data/인터뷰/*.json")

In [15]:
for file in file_list:
    # os 라이브러리를 이용하여 파일의 목록을 불러왔을때
    path = file_path + file
    print(path)
    break

../data/인터뷰/ckmk_d_bm_f_n_169087.json


In [16]:
# answer의 원문과 요약 텍스트들을 저장할수 있는 빈 리스트 생성
answers, summarys = [], []
for file in file_list2:
    # print(file)
    # break
    try:
        with open(file, 'r', encoding='utf-8') as f:
            dict_data = json.load(f)
            answer = dict_data['dataSet']['answer']['raw']['text']
            summary = dict_data['dataSet']['answer']['summary']['text']
            answers.append(answer)
            summarys.append(summary)
    except Exception as e:
        print(e)
        continue

Invalid control character at: line 23 column 135 (char 755)
Invalid control character at: line 23 column 571 (char 1132)
Invalid control character at: line 23 column 433 (char 1029)


In [17]:
len(answers)

12801

In [18]:
# 학습에 걸리는 시간이 길기때문에 일부분만 확인
train_docs = answers[ : 100]
train_sums = summarys[ : 100]
valid_docs = answers[-10 : ]
valid_sums = summarys[-10 : ]

In [19]:
print( len(train_docs), len(train_sums) )
print( len(valid_docs), len(valid_sums) )

100 100
10 10


In [20]:
import numpy as np 
from datasets import Dataset, DatasetDict
import evaluate

from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments
)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
model_name = "gogamza/kobart-base-v2"

In [22]:
# DatasetDict 생성 
raw_ds = DatasetDict(
    {
        'train' : Dataset.from_dict(
            {
                'document' : train_docs, 
                'summary' : train_sums
            }
        ), 
        'validation' : Dataset.from_dict(
            {
                'document' : valid_docs, 
                'summary' : valid_sums
            }
        )
    }
)

In [23]:
# 토크나이저, 모델 로드 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 입 출력 길이 설정 
max_input_len = 512
max_target_len = 256

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


In [24]:
def tok_fn(batch):
    inputs = tokenizer(
        batch['document'], 
        max_length = max_input_len, 
        padding = 'max_length', 
        truncation = True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['summary'], 
            max_length = max_target_len, 
            padding = 'max_length', 
            truncation = True
        )
    # labels의 input_ids에서 padding_token을 -100으로 변경 -> inputs['labels'] 대입
    labels_ids = np.array(labels['input_ids'])
    labels_ids[labels_ids == tokenizer.pad_token_id ] = -100
    inputs['labels'] = labels_ids.tolist()

    return inputs

In [25]:

tokenized_ds = raw_ds.map(
    tok_fn, batched= True, remove_columns= ['document', 'summary']
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 10/10 [00:00<00:00, 765.98 examples/s]


In [26]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer= tokenizer,
    model= model
)

In [ ]:
# metrics 지정
rouge = evaluate.load('rouge')

In [28]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # 디코더의 역할: vocab의 인덱스로 구성된 리스트를 다시 문자 형태로 변환
    #   예: preds = [13, 17, 102, ...(사전 vocab=['A', 'B', ...]에서 해당 값의 위치)] 처럼 labels 데이터와 같은 형태로 만들어질 것.
    # padding 토큰을 무시하기 위해 -100으로 구성했던 것을, 원래의 id 값으로 전환
    labels = np.where(
        labels != -100, labels, tokenizer.pad_token_id
    )
    # 텍스트 디코딩
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)  # 특수 토큰들은 디코드하지 않음
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 문장의 좌우에 공백이 존재하는 경우 다른 값으로 측정하기 때문에 좌우 공백 제거
    pred_str = [ doc.strip() for doc in pred_str ]
    label_str = [ doc.strip() for doc in label_str ]

    # Rouge 계산
    result = rouge.compute(
        predictions= pred_str,
        references= label_str,
        use_stemmer= True       # 점수 계산 시 단어의 어간을 기준으로 비교할 것인가?
    )

    # Rouge를 보기 편한 형태로 변경
    result = { k: round(v * 100, 2) for k, v in result.items() }

    return result

In [29]:
# Trainer 파라미터 값을 지정 
args = Seq2SeqTrainingArguments(
    output_dir= "./kobart", 
    eval_strategy= 'epoch', 
    save_strategy= 'epoch', 
    learning_rate= 5e-5, 
    num_train_epochs= 1, 
    logging_steps=10, 

    # generate 설정을 변경 
    predict_with_generate=True, 
    generation_max_length= 70, 
    # 요약 데이터를 생성할때 문장 후보의 탐색의 개수를 설정
    generation_num_beams= 4, 

    load_best_model_at_end= True, 
    metric_for_best_model='rougeL', 
    greater_is_better=True, 
    report_to=[]
)

In [ ]:
# Trainer 생성 
trainer = Seq2SeqTrainer(
    model = model, 
    args=args, 
    train_dataset= tokenized_ds['train'], 
    eval_dataset=tokenized_ds['validation'], 
    tokenizer = tokenizer, 
    data_collator= data_collator, 
    compute_metrics= compute_metrics
)
trainer.train()

C:\Users\student\AppData\Local\Temp\ipykernel_14320\780977535.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


In [ ]:
test_text = "과학기술정보통신부는 초거대 AI 연구 인프라 지원을 강화한다고 밝혔다. 스타트업 새당으로 GPU 리소스를 확대 제공할 계획이다."

inputs = tokenizer(
    test_text,
    return_tensors = 'pt',
    truncation = True,
    max_length = max_input_len
)
inputs.pop('token_type_ids', None)

# 이 부분을 바꾸는 게 중요
gen_ids = model.generate(
    **inputs.to(model.device),
    max_new_tokens = 50,                    # 출력 토큰의 길이
    min_new_tokens = 10,                    # 출력 토큰의 최소 길이
    num_beams = 4,                          # n개의 후보 문장을 병렬적으로 추적하여 가장 가능성이 높은 문장을 선택
    do_sample = False                       # 샘플링 사용 여부 (False: 빔 서치, True: 랜덤하게 선택)
    length_penalty = 1.6,                   # 생성 길이에 대한 가중치 (반복적으로 요약문이 나오)
    no_repeat_ngram_size = 5,               # 반복 방지
    repetition_penalty = 1.5,               # 토큰 반복 패턴이 나타나는 경우 페널티 적용
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.pad_token_id   # 안정성을 위한 옵션
)

print(tokenizer.decode(gen_ids[0], skip_special_tokens=True))